In [ ]:
import os
from pathlib import Path
from gurobipy import Model, GRB
from data_loader import load_prices, load_storage, load_conversion, load_plant_capacity, load_demand

# Data Loading from data folder in the parent directory
current_dir = Path().resolve()
data_folder = os.path.join(current_dir.parent, "data")
prices_file = os.path.join(data_folder, "prices.csv")           # Price in EUR/tonne (coal,oil) or EUR/m3 (gas)
storage_file = os.path.join(data_folder, "storage.csv")         # Max storage in tonnes/m3
conversion_file = os.path.join(data_folder, "conversion.csv")   # Conversion rates in MWh/tonne(m3)
plant_file = os.path.join(data_folder, "plant_capacity.csv")    # Plant capacity in MWh/month
demand_file = os.path.join(data_folder, "demand.csv")           # Monthly demand in MWh

try:
    prices_df = load_prices(prices_file)
    storage = load_storage(storage_file)
    conversion = load_conversion(conversion_file)
    plant_capacity = load_plant_capacity(plant_file)
    demand_df = load_demand(demand_file)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")


# Model Initialization

m = Model("Fuel_Procurement_OneMonth")

# Decision Variables
x = {fuel: m.addVar(lb=0, ub=storage[fuel], name=f"buy_{fuel}") for fuel in prices_df.columns if fuel != "month"}
y = {fuel: m.addVar(lb=0, ub=plant_capacity[fuel], name=f"gen_{fuel}") for fuel in prices_df.columns if fuel != "month"}

m.update()

# Constraints
m.addConstr(sum(y[fuel] for fuel in y) >= demand_df.iloc[0]["demand"], "Demand_Constraint")
for fuel in y:
    m.addConstr(y[fuel] <= conversion[fuel] * x[fuel], f"Conversion_Constraint_{fuel}")
m.update()

# Objective
m.setObjective(sum(prices_df.iloc[0][fuel] * x[fuel] for fuel in x),GRB.MINIMIZE) 
m.update()

# Optimize
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    for fuel in x:
        print(f"Purchased {x[fuel].X} units of {fuel}")
    for fuel in y:
        print(f"Generated {y[fuel].X} units of electricity from {fuel}")
    for fuel in x:
        cost = prices_df.iloc[0][fuel] * x[fuel].X
        print(f"Cost for {fuel}: EUR {cost}")
    total_cost = m.objVal
    print(f"Total Cost: EUR {total_cost}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 6 columns and 9 nonzeros
Model fingerprint: 0x5ad0c8ae
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+01, 9e+01]
  Bounds range     [4e+02, 5e+03]
  RHS range        [1e+03, 1e+03]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.250000e+02   0.000000e+00      0s
       1    9.1034483e+03   0.000000e+00   0.000000e+00      0s



Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.103448276e+03
Optimal Solution Found:
Purchased 0.0 units of coal
Purchased 85.71428571428571 units of gas
Purchased 34.48275862068966 units of oil
Generated 0.0 units of electricity from coal
Generated 600.0 units of electricity from gas
Generated 400.0 units of electricity from oil
Cost for coal: EUR 0.0
Cost for gas: EUR 6000.0
Cost for oil: EUR 3103.4482758620693
Total Cost: EUR 9103.448275862069
